# Imports


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import os
import community as community_louvain
from networkx.algorithms.community import greedy_modularity_communities


# Utility Functions

In [1]:
def remove_rows_by_column_value(df, column_name, value_to_remove):
    """
    Remove all rows from a DataFrame where a specific column matches a given value.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to filter.
        value_to_remove (str): The value to remove from the column.

    Returns:
        pd.DataFrame: A new DataFrame with the specified rows removed.

    Raises:
        KeyError: If the specified column is not in the DataFrame.
    """
    if column_name not in df.columns:
        raise KeyError(f"Column '{column_name}' not found in DataFrame.")

    filtered_df = df[df[column_name] != value_to_remove].copy()
    return filtered_df

def remove_duplicate_rows(df):
    """
    Remove duplicate rows from a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame without duplicate rows.
    """
    cleaned_df = df.drop_duplicates().copy()
    return cleaned_df

def sample_dataframe(df, percentage, random_state=None):
    """
    Randomly sample a percentage of rows from a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        percentage (float): Percentage of rows to sample (between 0 and 100).
        random_state (int, optional): Seed for reproducibility.

    Returns:
        pd.DataFrame: A sampled subset of the original DataFrame.

    Raises:
        ValueError: If percentage is not between 0 and 100.
    """
    if not (0 < percentage <= 100):
        raise ValueError("Percentage must be between 0 and 100.")

    frac = percentage / 100.0
    sampled_df = df.sample(frac=frac, random_state=random_state).copy()
    return sampled_df

def stratified_sample_by_column(df, column, percentage, random_state=None):
    """
    Reduce a DataFrame to a given percentage while preserving the distribution
    of values in a specific column (stratified sampling).

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column (str): Column name to preserve distribution (e.g., "Cuisine").
        percentage (float): Percentage of total rows to keep (0 < percentage <= 100).
        random_state (int, optional): Seed for reproducibility.

    Returns:
        pd.DataFrame: A new stratified-sampled DataFrame.

    Raises:
        ValueError: If percentage is not between 0 and 100.
        KeyError: If the specified column does not exist.
    """
    if not (0 < percentage <= 100):
        raise ValueError("Percentage must be between 0 and 100.")
    if column not in df.columns:
        raise KeyError(f"Column '{column}' not found in DataFrame.")

    sampled_dfs = []
    frac = percentage / 100.0

    for group_value, group_df in df.groupby(column):
        group_sample = group_df.sample(frac=frac, random_state=random_state)
        sampled_dfs.append(group_sample)

    result_df = pd.concat(sampled_dfs).reset_index(drop=True)
    return result_df

def calculate_network_descriptors(G):
    """
    Compute key structural and macroscopic descriptors for the given network.

    Parameters:
        G (networkx.Graph): The input graph.

    Returns:
        dict: A dictionary containing the following metrics:
            - 'nodes': Number of nodes in the network.
            - 'edges': Number of edges in the network.
            - 'self_loops': Number of self-loops.
            - 'multi_edges': Number of multi-edges.
            - 'degree_min': Minimum node degree.
            - 'degree_max': Maximum node degree.
            - 'degree_avg': Average node degree.
            - 'clustering': Average clustering coefficient.
            - 'assortativity': Degree assortativity coefficient.
            - 'avg_path_length': Average shortest path length (largest connected component).
            - 'diameter': Diameter of the largest connected component.
    """
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    degrees = [d for _, d in G.degree()]

    self_loops = nx.number_of_selfloops(G)
    multi_edges = sum(1 for edge in G.edges() if G.number_of_edges(edge[0], edge[1]) > 1)

    largest_cc = max(nx.connected_components(G), key=len)
    g_largest_cc = G.subgraph(largest_cc)

    return {
        'nodes': n_nodes,
        'edges': n_edges,
        'self_loops': self_loops,
        'multi_edges': multi_edges,
        'degree_min': np.min(degrees),
        'degree_max': np.max(degrees),
        'degree_avg': np.mean(degrees),
        'clustering': nx.average_clustering(G),
        'assortativity': nx.degree_assortativity_coefficient(G),
        'avg_path_length': nx.average_shortest_path_length(g_largest_cc),
        'diameter': nx.diameter(g_largest_cc),
    }

def calculate_centralities(G):
    """
    Compute node centrality measures for a single network.

    Parameters:
        G (networkx.Graph): The input graph.

    Returns:
        - dict: A dictionary with centrality measures:
            - 'betweenness': Betweenness centrality for each node.
            - 'degree': Degree centrality for each node.
            - 'eigenvector': Eigenvector centrality for each node.
    """
    return {
        'betweenness': nx.betweenness_centrality(G),
        'degree': dict(G.degree()),
        'eigenvector': nx.eigenvector_centrality(G)
    }

def plot_degree_distribution(degree_sequence, net_name, save=True, show=False, out_dir='report'):
    """
    Plot degree distribution in both linear and logarithmic scales.

    Parameters:
        degree_sequence (list): List of node degrees.
        net_name (str): Network name for title and filename.
        save (bool): If True, saves the plot as PNG in the output directory.
        show (bool): If True, displays the plot.
        out_dir (str): Output directory for saving the plot.
    """
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    degree_counts = Counter(degree_sequence)
    min_degree = min(degree_sequence)
    max_degree = max(degree_sequence)

    degree = list(range(min_degree, max_degree + 1))
    degree_count = [degree_counts.get(x, 0) for x in degree]

    # Linear scale
    ax[0].scatter(degree, degree_count, color='blue', label='data')
    ax[0].set_xlabel('$k$', fontsize=14)
    ax[0].set_ylabel('$P(k)$', fontsize=14)
    ax[0].set_title('Linear scale', fontsize=14)
    ax[0].tick_params(axis='both', labelsize=12)

    # Log-log scale
    ax[1].scatter(degree, degree_count, color='red', label='data')
    ax[1].set_xlabel('$k$', fontsize=14)
    ax[1].set_ylabel('$P(k)$', fontsize=14)
    ax[1].set_xscale('log')
    ax[1].set_yscale('log')
    ax[1].set_title('Logarithmic scale', fontsize=14)
    ax[1].tick_params(axis='both', labelsize=12)

    plt.suptitle(f'Degree Distribution: {net_name}', fontsize=16)
    plt.tight_layout()

    if save:
        os.makedirs(out_dir, exist_ok=True)
        file_path = os.path.join(out_dir, f"{net_name}_degree_distribution.png")
        plt.savefig(file_path, bbox_inches='tight')

    if show:
        plt.show()

    plt.close()

def add_bipartite_nodes_from_dataframe(G, df, column_id, bipartite, prefix):
    """
    Add nodes to a graph from a DataFrame for a bipartite graph construction.

    Parameters:
        G (networkx.Graph): The graph to which nodes are added.
        df (pd.DataFrame): The DataFrame containing node data.
        column_id (str): The name of the column with unique node identifiers.
        bipartite (int): The bipartite set this node belongs to (0 or 1).
        prefix (str): Prefix to prepend to the node ID (e.g., 'R' for recipes, 'I' for ingredients).

    Returns:
        None (the graph is modified in-place).
    """
    if column_id not in df.columns:
        raise KeyError(f"Column '{column_id}' not found in DataFrame.")

    for _, row in df.iterrows():
        node_id = f"{prefix}{row[column_id]}"
        attrs = row.drop(column_id).to_dict()
        attrs['bipartite'] = bipartite
        G.add_node(node_id, **attrs)

def apply_greedy_community_detection(G, weight=None):
    """
    Applies the Greedy Modularity algorithm to the given graph.

    Parameters:
        G (networkx.Graph): The graph on which to run community detection.
        weight (str or None): Edge attribute to use as weight. If None, the graph is unweighted.

    Returns:
        list: A list of sets, where each set contains the nodes in one community.
    """
    return list(greedy_modularity_communities(G, weight=weight))


def detect_communities_louvain(G):
    """
    Apply Louvain algorithm to detect communities in G using NetworkX interface.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    return list(nx.community.louvain_communities(G))

def add_edges_from_dataframe(G, df, col1, col2, prefix1, prefix2):
    """
    Add edges to a graph from a DataFrame using two columns and respective prefixes.

    Parameters:
        G (networkx.Graph): The graph to which edges are added.
        df (pd.DataFrame): The DataFrame containing edge data.
        col1 (str): Column name for the first node ID (e.g., "Recipe ID").
        col2 (str): Column name for the second node ID (e.g., "Entity ID").
        prefix1 (str): Prefix for nodes from col1 (e.g., "R").
        prefix2 (str): Prefix for nodes from col2 (e.g., "I").

    Returns:
        None (the graph is modified in-place).
    """
    if col1 not in df.columns or col2 not in df.columns:
        raise KeyError(f"Columns '{col1}' and/or '{col2}' not found in DataFrame.")

    for _, row in df.iterrows():
        node1 = f"{prefix1}{row[col1]}"
        node2 = f"{prefix2}{row[col2]}"
        G.add_edge(node1, node2)